## **Python for Data Engineers**
### ***Advanced ETL techniques for beginners***
[Scripts source](https://towardsdatascience.com/python-for-data-engineers-f3d5db59b6dd)

### **2. Generators**

>> **Often we need to validate data files before we load them into a data warehouse. If one record fails the whole batch will fail.**

In [4]:
import csv
import urllib.request  # Import the urllib.request module for Python 3

# assigning url to a variable
url="https://raw.githubusercontent.com/Tanishqa-10/AskPython/main/Sampledata.csv"


# Use urllib.request.urlopen() to fetch the URL content
response = urllib.request.urlopen(url)

# Read the CSV content using csv.reader()
cr = csv.reader(response.read().decode('utf-8').splitlines())

# Loop through each row and print it
for row in cr:
    print(row)

['Name', 'Last Name', 'Member', 'City', 'Code', 'Amount']
['Jim', 'Doe', 'No', 'Ohio', ' JD', '8075']
['Gill', 'McGinnis', 'Yes', 'Texas', ' GM', '9119']
['Joe', 'Repici', 'Yes', 'Bostan', ' JR', '8075']
['Stephen', 'Tyler', 'No', 'LA', 'ST', '91234']
['Oscar', 'Blankman', 'No', 'NYC', 'OB', '298']
['Kelly', 'Jet', ' No', 'Desert City', 'KJ', '123']


In [6]:
type(cr)

_csv.reader

In [21]:
import pandas as pd
df = pd.read_csv(url)
display(df)
df.to_csv('very_big_file.csv')

Name Last Name Member         City Code  Amount
0      Jim       Doe     No         Ohio   JD    8075
1     Gill  McGinnis    Yes        Texas   GM    9119
2      Joe    Repici    Yes       Bostan   JR    8075
3  Stephen     Tyler     No           LA   ST   91234
4    Oscar  Blankman     No          NYC   OB     298
5    Kelly       Jet     No  Desert City   KJ     123

In [15]:
def validate_schema(data):
    # insert them into a data warehouse table for batch data processing pipelines.
    return "Injection Successfully done"

In [16]:
for line in open('very_big_file.csv'):
    validate_schema(line)

# or the same using list comprehension:
data_errors = [validate_schema(line) for line in open('very_big_file.csv')]

In [17]:
print(data_errors)

['Injection Successfully done', 'Injection Successfully done', 'Injection Successfully done', 'Injection Successfully done', 'Injection Successfully done', 'Injection Successfully done', 'Injection Successfully done']


>> The data is processed, extracted, and transformed in steps. Therefore, a sequence of data processing stages can be referred to as a **data pipeline**.

>> So it is a sequence of data processing steps. Due to logical data flow connections between these stages, each stage generates an output that serves as an input for the following stage.




## **data pipeline when data is processed between source and destination**

<img src= "https://miro.medium.com/v2/resize:fit:786/format:webp/0*b0yA7vS3Fg4nP9_P.png">


## **Logical structure of a data warehouse pipeline**
<img src="https://miro.medium.com/v2/resize:fit:786/format:webp/0*7GhrdwcLpC4JUrlU.png">


## **Logical structure of a data lake pipeline**
<img src ="https://miro.medium.com/v2/resize:fit:1400/format:webp/0*uVLXwx6E0CmVAS21.png">

아래 파이썬 스크립트는 큰 CSV 파일을 청크 단위로 읽어들이고 각 데이터 청크에 ETL 작업을 적용하는 거야. 우리가 만든 etl() 함수는 "Jim"을 "****"로 바꿔주고, batch_read_file() 함수는 파일을 청크 단위로 읽어들이는 거야.

그러니까, 이 스크립트를 실행하면 very_big_file.csv 파일을 청크 단위로 읽어들여서 각 청크에 ETL 작업을 적용하고, 수정된 데이터를 출력해.

In [26]:
# Create a file first: ./very_big_file.csv as:
# transaction_id,user_id,total_cost,dt
# 1,John,10.99,2023-04-15
# 2,Mary, 4.99,2023-04-12


# Example.py
def etl(item):
    # Do some etl here
    return item.replace("Jim", '****')

# Create a generator
def batch_read_file(file_object, batch_size=10):
    """Lazy function (generator) can read a file in chunks.
    Default chunk: 1024 bytes."""
    while True:
        data = file_object.read(batch_size)
        if not data:
            break
        yield data

# and read in chunks
# read a local file and process it in chunks of batch_size bytes.
with open('very_big_file.csv') as f:
    for batch in batch_read_file(f):
        print(etl(batch))

# In command line run
# Python example.py

,Name,Last
 Name,Memb
er,City,Co
de,Amount

0,****,Doe,
No,Ohio, J
D,8075
1,G
ill,McGinn
is,Yes,Tex
as, GM,911
9
2,Joe,Re
pici,Yes,B
ostan, JR,
8075
3,Ste
phen,Tyler
,No,LA,ST,
91234
4,Os
car,Blankm
an,No,NYC,
OB,298
5,K
elly,Jet, 
No,Desert 
City,KJ,12
3



### 클라우드 저장소에서 텍스트 데이터를 가져오면서 스트림으로 처리할 수 있어. 전체 데이터 파일을 로드하고 한 줄씩 처리하는 건 메모리를 많이 소모하잖아. 그래서 re.finditer를 사용하면 돼. 이건 제너레이터처럼 작동해서 데이터 파일을 청크 단위로 읽을 수 있어서 필요한 ETL을 할 때 메모리를 너무 많이 소비하지 않아.

In [27]:
import io
import re
def etl(item):
    print(f'Transforming item: {item}')
    return item.replace("Jim", '****')

# Helper function to split our text file into chunks
# using separator
def splitStr(string, sep="\s+"):
    if sep=='':
        return (c for c in string)
    else:
        return (_.group(1) for _ in re.finditer(f'(?:^|{sep})((?:(?!{sep}).)*)', string))

# Text file loaded as a blob
blob = """transaction_id,user_id,total_cost,dt
1,John,10.99,2023-04-15
2,Mary, 4.99,2023-04-12
"""

# data = blob.split("\n") # We wouldn't want to do this on large datasets
# as it would require to load big data file as a whole in the first place
# consuming lots of memory

# We would want to use our generator helper function
# and process data in chunks
data = splitStr(blob, sep='\n')
data_str = u"\n".join(etl(item) for item in data)

print('New file contents:')
print(data_str)
data_file = io.BytesIO(data_str.encode())

print('This data file is ready for BigQuery:')
print(data_file)

Transforming item: transaction_id,user_id,total_cost,dt
Transforming item: 1,John,10.99,2023-04-15
Transforming item: 2,Mary, 4.99,2023-04-12
Transforming item: 
New file contents:
transaction_id,user_id,total_cost,dt
1,John,10.99,2023-04-15
2,Mary, 4.99,2023-04-12

This data file is ready for BigQuery:


## **End**